# 01 – Orders landing & basic cleaning

## Purpose
Load raw order export and perform **minimal, non-semantic cleaning**:
- remove unused columns
- normalize text fields
- clean numeric fields (prices)
- preserve original quantities and product descriptions

⚠️ This notebook **does NOT**:
- explode bundled products
- infer product categories
- derive per-unit quantities
- aggregate by customer, day, or product group

All semantic interpretation happens in later notebooks.


In [21]:
import pandas as pd

raw_data = pd.read_csv("../data/raw/full_orders_data.csv")

raw_data.columns

/var/folders/2w/84n_l6ld0kq5y9sp9jw4q50c0000gn/T/ipykernel_32222/231078115.py:3: DtypeWarning: Columns (1,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv("../data/raw/full_orders_data.csv")


Index(['kolejności', 'Kwota', 'Produkty', 'Kod', 'Miejscowość', 'Anon',
       'Ilość zakupów', 'Data zakupu', 'Miesiąc zakupu', 'Źródło', 'Kolejność',
       'Kolejność 2', 'Butelka filtrująca', 'Dzbanek', 'Filtry dzbankowe',
       'Filtry butelkowe', 'SeeYoo', 'Akcesoria butelkowe', 'Termiczne',
       'Limitowane', 'Flow Comfort - start', 'Flow Comfort - wkład',
       'Filtr pokipropylenowy', 'Bidon', 'Syfon', 'Suplement',
       'Wymiana Butli'],
      dtype='object')

In [22]:
pd.set_option("display.max_colwidth", None)

In [23]:
print(len(raw_data))

raw_data.head()

416082


,kolejności,Kwota,Produkty,Kod,Miejscowość,Anon,Ilość zakupów,Data zakupu,Miesiąc zakupu,Źródło,...,Akcesoria butelkowe,Termiczne,Limitowane,Flow Comfort - start,Flow Comfort - wkład,Filtr pokipropylenowy,Bidon,Syfon,Suplement,Wymiana Butli
0,68640194,87.99,"Butelka filtrująca Dafi SOLID 0,7 l szafirowa + filtr węglowy (x1)Rurka na filtr do butelki filtrującej Dafi SOLID 0,7 l szafirowym (x1)ZESTAW 3 filtry do butelki filtrującej Dafi SOFT i SOLID szafirowy (x1)",44-213,RYBNIK,ANON_0000001,1,11/9/22,22_11_LIS,DAFI,...,Akcesoria butelkowe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,68665043,59.98,"Butelka filtrująca Dafi SOFT 0,5 l limonkowa + filtr węglowy (x1)Butelka filtrująca Dafi SOFT 0,5 l niebiańska + filtr węglowy (x1)",11-111,Test,ANON_0000002,1,11/9/22,22_11_LIS,DAFI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,68689466,199.0,"Dafi przepływowy podgrzewacz wody nadumywalkowy 3,7 KW z baterią białą (x1)",24-200,Bełżyce,ANON_0000003,1,11/9/22,22_11_LIS,DAFI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,68689468,35.0,"Zakrętka do butelki filtrującej Dafi SOLID uchwyt flamingowy (x1)Rurka na filtr do butelki filtrującej Dafi SOLID 0,5 l flamingowa (x1)Rurka na filtr do butelki filtrującej Dafi SOLID 0,5 l turkusowa (x1)",31-345,Krakow,ANON_0000004,1,11/9/22,22_11_LIS,DAFI,...,Akcesoria butelkowe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,68694434,29.99,ZESTAW 3 filtry do butelki filtrującej Dafi SOFT i SOLID turkusowy (x1),59-220,Legnica,ANON_0000005,1,11/9/22,22_11_LIS,DAFI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
drop = ['Miesiąc zakupu', 'Kolejność',
       'Kolejność 2', 'Butelka filtrująca', 'Dzbanek', 'Filtry dzbankowe',
       'Filtry butelkowe', 'SeeYoo', 'Akcesoria butelkowe', 'Termiczne',
       'Limitowane', 'Flow Comfort - start', 'Flow Comfort - wkład',
       'Filtr pokipropylenowy', 'Bidon', 'Syfon', 'Suplement',
       'Wymiana Butli']

short = raw_data.drop(columns=drop)

short.head()

,kolejności,Kwota,Produkty,Kod,Miejscowość,Anon,Ilość zakupów,Data zakupu,Źródło
0,68640194,87.99,"Butelka filtrująca Dafi SOLID 0,7 l szafirowa + filtr węglowy (x1)Rurka na filtr do butelki filtrującej Dafi SOLID 0,7 l szafirowym (x1)ZESTAW 3 filtry do butelki filtrującej Dafi SOFT i SOLID szafirowy (x1)",44-213,RYBNIK,ANON_0000001,1,11/9/22,DAFI
1,68665043,59.98,"Butelka filtrująca Dafi SOFT 0,5 l limonkowa + filtr węglowy (x1)Butelka filtrująca Dafi SOFT 0,5 l niebiańska + filtr węglowy (x1)",11-111,Test,ANON_0000002,1,11/9/22,DAFI
2,68689466,199.0,"Dafi przepływowy podgrzewacz wody nadumywalkowy 3,7 KW z baterią białą (x1)",24-200,Bełżyce,ANON_0000003,1,11/9/22,DAFI
3,68689468,35.0,"Zakrętka do butelki filtrującej Dafi SOLID uchwyt flamingowy (x1)Rurka na filtr do butelki filtrującej Dafi SOLID 0,5 l flamingowa (x1)Rurka na filtr do butelki filtrującej Dafi SOLID 0,5 l turkusowa (x1)",31-345,Krakow,ANON_0000004,1,11/9/22,DAFI
4,68694434,29.99,ZESTAW 3 filtry do butelki filtrującej Dafi SOFT i SOLID turkusowy (x1),59-220,Legnica,ANON_0000005,1,11/9/22,DAFI


In [25]:
'''Simple normalization'''


import re 

def normalize_text(s):
    if pd.isna(s):
        return ""
    s = str(s).lower()
    s=re.sub(r"\s+", " ", s)
    return s.strip()

df = short

df["produkty_clean"] = df["Produkty"].apply(normalize_text)

df[["Produkty", "produkty_clean"]].head(3)

,Produkty,produkty_clean
0,"Butelka filtrująca Dafi SOLID 0,7 l szafirowa + filtr węglowy (x1)Rurka na filtr do butelki filtrującej Dafi SOLID 0,7 l szafirowym (x1)ZESTAW 3 filtry do butelki filtrującej Dafi SOFT i SOLID szafirowy (x1)","butelka filtrująca dafi solid 0,7 l szafirowa + filtr węglowy (x1)rurka na filtr do butelki filtrującej dafi solid 0,7 l szafirowym (x1)zestaw 3 filtry do butelki filtrującej dafi soft i solid szafirowy (x1)"
1,"Butelka filtrująca Dafi SOFT 0,5 l limonkowa + filtr węglowy (x1)Butelka filtrująca Dafi SOFT 0,5 l niebiańska + filtr węglowy (x1)","butelka filtrująca dafi soft 0,5 l limonkowa + filtr węglowy (x1)butelka filtrująca dafi soft 0,5 l niebiańska + filtr węglowy (x1)"
2,"Dafi przepływowy podgrzewacz wody nadumywalkowy 3,7 KW z baterią białą (x1)","dafi przepływowy podgrzewacz wody nadumywalkowy 3,7 kw z baterią białą (x1)"


In [26]:
df.head()

,kolejności,Kwota,Produkty,Kod,Miejscowość,Anon,Ilość zakupów,Data zakupu,Źródło,produkty_clean
0,68640194,87.99,"Butelka filtrująca Dafi SOLID 0,7 l szafirowa + filtr węglowy (x1)Rurka na filtr do butelki filtrującej Dafi SOLID 0,7 l szafirowym (x1)ZESTAW 3 filtry do butelki filtrującej Dafi SOFT i SOLID szafirowy (x1)",44-213,RYBNIK,ANON_0000001,1,11/9/22,DAFI,"butelka filtrująca dafi solid 0,7 l szafirowa + filtr węglowy (x1)rurka na filtr do butelki filtrującej dafi solid 0,7 l szafirowym (x1)zestaw 3 filtry do butelki filtrującej dafi soft i solid szafirowy (x1)"
1,68665043,59.98,"Butelka filtrująca Dafi SOFT 0,5 l limonkowa + filtr węglowy (x1)Butelka filtrująca Dafi SOFT 0,5 l niebiańska + filtr węglowy (x1)",11-111,Test,ANON_0000002,1,11/9/22,DAFI,"butelka filtrująca dafi soft 0,5 l limonkowa + filtr węglowy (x1)butelka filtrująca dafi soft 0,5 l niebiańska + filtr węglowy (x1)"
2,68689466,199.0,"Dafi przepływowy podgrzewacz wody nadumywalkowy 3,7 KW z baterią białą (x1)",24-200,Bełżyce,ANON_0000003,1,11/9/22,DAFI,"dafi przepływowy podgrzewacz wody nadumywalkowy 3,7 kw z baterią białą (x1)"
3,68689468,35.0,"Zakrętka do butelki filtrującej Dafi SOLID uchwyt flamingowy (x1)Rurka na filtr do butelki filtrującej Dafi SOLID 0,5 l flamingowa (x1)Rurka na filtr do butelki filtrującej Dafi SOLID 0,5 l turkusowa (x1)",31-345,Krakow,ANON_0000004,1,11/9/22,DAFI,"zakrętka do butelki filtrującej dafi solid uchwyt flamingowy (x1)rurka na filtr do butelki filtrującej dafi solid 0,5 l flamingowa (x1)rurka na filtr do butelki filtrującej dafi solid 0,5 l turkusowa (x1)"
4,68694434,29.99,ZESTAW 3 filtry do butelki filtrującej Dafi SOFT i SOLID turkusowy (x1),59-220,Legnica,ANON_0000005,1,11/9/22,DAFI,zestaw 3 filtry do butelki filtrującej dafi soft i solid turkusowy (x1)


In [27]:
# Clean Kwota to numeric
kw = df["Kwota"].astype(str).str.strip()

kw = (
    kw.str.replace("\u00a0", "", regex=False)   # non-breaking spaces
      .str.replace(" ", "", regex=False)        # normal spaces
      .str.replace(",", ".", regex=False)       # decimal commas
      .str.replace(r"[^\d\.\-]", "", regex=True)  # remove currency/text
)

df["Kwota"] = pd.to_numeric(kw, errors="coerce")


In [28]:
# -------------------------
# Sanity checks
# -------------------------
print("Rows:", len(df))
print("Unique customers:", df["Anon"].nunique())
print("Missing dates:", df["Data zakupu"].isna().mean())
print("Missing quantities:", df["Ilość zakupów"].isna().mean())
print("Top quantity values:")
print(df["Ilość zakupów"].value_counts().head(10))


Rows: 416082
Unique customers: 336612
Missing dates: 0.0
Missing quantities: 0.0
Top quantity values:
Ilość zakupów
1     282517
2      77972
3      28836
4      12532
5       6440
6       3096
7       1848
8       1024
9        594
10       340
Name: count, dtype: int64


In [29]:
df.to_csv("../data/cleaned/orders_clean.csv")

df.to_parquet("../data/interim/orders_landing.parquet")

## Output contract: `orders_landing.parquet`

**Grain**  
One row per **order line as present in the raw export** (may include bundles).

**Key columns**
- `Anon` – anonymized customer ID
- `Data zakupu` – purchase date (string, day-level)
- `Produkty` – raw product description (may include multiple items)
- `produkty_clean` – normalized text version of `Produkty`
- `Ilość zakupów` – quantity as reported in the raw data (**not per-unit**)
- `Kwota` – numeric order line value

⚠️ Quantities in this table should be treated as **raw inputs**, not final unit counts.
